In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from glob import glob
from configs import *
from tqdm import tqdm
from utils import *
import numpy as np
import matplotlib.pyplot as plt
from dataset import *
tqdm.monitor_interval = 0

In [3]:
FORECAST = 5

In [4]:
dset = Routes('train', 32, index_file='min-data.json')
evalset = Routes('test', 32, index_file='min-data.json')

Routes dataset: train
 [*] Loaded routes: 224 (0.29s)
 [*] Subset train: 204 (reserved_routes.json)
 [*] Loaded trainable inds: 240935 (0.22s)
Routes dataset: test
 [*] Loaded routes: 224 (0.32s)
 [*] Subset test: 20 (reserved_routes.json)
 [*] Loaded trainable inds: 54181 (0.02s)


In [5]:
import torch
import torch.nn as nn
import numpy as np

In [6]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [10]:
from models.Linear import Linear

In [11]:
model = Linear(forecast=FORECAST).to(device)

In [12]:
criterion, opt, sch = model.params(lr=0.0001)

In [21]:
def evaluate():
    eval_losses = []
    for bii in tqdm(range(evalset.size())):
        model.eval()
        
        Xs, Ys = evalset.next()
        Xs, Ys = model.format_batch(Xs, Ys, gpu=device)

        outputs = model(Xs)
        loss = criterion(outputs, Ys)
        eval_losses.append(loss.item())
    print('Eval loss:', np.mean(eval_losses))

In [22]:
evaluate()

100%|██████████| 1693/1693 [01:34<00:00, 17.91it/s]

Eval loss: 33.33506097697818


In [24]:
EPS = 5
losses = []
for eii  in range(EPS):
    sch.step()
    for bii in range(dset.size()):
        model.train()
        
        Xs, Ys = dset.next()
        Xs, Ys = model.format_batch(Xs, Ys, gpu=device)

        outputs = model(Xs)

        opt.zero_grad()
        loss = criterion(outputs, Ys)
        loss.backward()
        losses.append(loss.item())
        opt.step()

        sys.stdout.write('[%d/%d : %d/%d] - L%.2f      \r' % (
            eii+1, EPS,
            bii+1, dset.size(), 
            loss.item()
        ))
    evaluate()
    torch.save(model, 'checkpoints/linear_e%d.pth' % (eii+1))
    sys.stdout.flush()

ValueError: too many values to unpack (expected 2)